<h1>Gaussian Elimination</h1>

<p>The solving of &#36;Ax&#61;b&#36; is typically taught first by using Gaussian Elimination. Rather than find &#36;LU&#36; and then solve, we reduce the augmented matrix &#36;&#91;A | b&#93;&#36; to an upper triangular form and then back substitute.</p>

<p>Let&#39;s see that we get &#36;U&#36; and &#36;L&#36; from this familar process:</p>

<h2>Example</h2>

<p>Following the text, let</p>

In [ ]:
A = [6 -2 -2 4; 12 -8 6 10; 3 -13 9 3; -6 4 1 -18]

4x4 Array{Int64,2}:
  6   -2  -2    4
 12   -8   6   10
  3  -13   9    3
 -6    4   1  -18

<p>and</p>

In [ ]:
b = [12, 34, 27, -38]

4-element Array{Int64,1}:
  12
  34
  27
 -38

<p>Then we form the augmented matrix, <code>M</code>:</p>

In [ ]:
M = [A b//1]

4x5 Array{Rational{Int64},2}:
  6//1   -2//1  -2//1    4//1   12//1
 12//1   -8//1   6//1   10//1   34//1
  3//1  -13//1   9//1    3//1   27//1
 -6//1    4//1   1//1  -18//1  -38//1

<p>We want to subtract the first row times a factor from the each of the next three rows:</p>

In [ ]:
M[2,:] = M[2,:] - 2*M[1,:]
M[3,:] = M[3,:] - 1//2*M[1,:]
M[4,:] = M[4,:] + 1*M[1,:]
M

4x5 Array{Rational{Int64},2}:
 6//1   -2//1  -2//1    4//1   12//1
 0//1   -4//1  10//1    2//1   10//1
 0//1  -12//1  10//1    1//1   21//1
 0//1    2//1  -1//1  -14//1  -26//1

<p>Then we look at the second row. We use that &#36;M_&#123;22&#125;&#36; value to &quot;knock out&quot; those below it:</p>

In [ ]:
M[3,:] = M[3,:] -3*M[2,:]
M[4,:] = M[4,:] +1//2 * M[2,:]
M

4x5 Array{Rational{Int64},2}:
 6//1  -2//1   -2//1    4//1   12//1
 0//1  -4//1   10//1    2//1   10//1
 0//1   0//1  -20//1   -5//1   -9//1
 0//1   0//1    4//1  -13//1  -21//1

<p>Finally, we use the third row to knock out the 4th:</p>

In [ ]:
M[4,:] = M[4,:] +1//5*M[3,:]

1x5 Array{Rational{Int64},2}:
 0//1  0//1  0//1  -14//1  -114//5

<p>We have &#36;U&#36; sitting in this:</p>

In [ ]:
U = M[:, 1:4]

4x4 Array{Rational{Int64},2}:
 6//1  -2//1   -2//1    4//1
 0//1  -4//1   10//1    2//1
 0//1   0//1  -20//1   -5//1
 0//1   0//1    0//1  -14//1

<p>And what about &#36;L&#36;? Form the matrix with the products:</p>

In [ ]:
L = [1 0 0 0;  2 1 0 0; 1//2 3 1 0; -1 -1//2 -1//5 1]

4x4 Array{Rational{Int64},2}:
  1//1   0//1   0//1  0//1
  2//1   1//1   0//1  0//1
  1//2   3//1   1//1  0//1
 -1//1  -1//2  -1//5  1//1

<p>We verify the product:</p>

In [ ]:
A - L*U

4x4 Array{Rational{Int64},2}:
 0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1

<h3>Where did &#36;L&#36; come from?</h3>

<p>We have the operations of adding &#36;\lambda&#36; to the &#36;i&#36;th row and adding to the &#36;j&#36;th row where &#36;j &gt; i&#36;. Call these &#36;R_m&#36;. And then we have:</p>

&#36;~
\begin&#123;align&#125;
&#91;U~ b&#39;&#93; &amp;&#61; R_n R_&#123;n-1&#125; \cdots R_2 R_1 &#91;A~ b&#93;\\
&#40;R_n R_&#123;n-1&#125; \cdots R_2 R_1&#41;^&#123;-1&#125; &#91; U ~ b&#39;&#93; &amp;&#61; &#91;A ~ b&#93; \\
&#91;LU ~ Lb&#39;&#93; &amp;&#61; &#91;A ~ b&#93;
\end&#123;align&#125;
~&#36;

<p>And we have &#36;LU &#61; A&#36;. Is &#36;L&#36; actually lower diagonal?</p>

<p>The &#36;R_m&#36; matrices look like &#36;I &#43; \lambda \delta_&#123;ij&#125;&#36;. The inverses for these are &#36;I - \delta^&#123;ij&#125;&#36;:</p>

&#36;~
&#40;I &#43; \lambda \delta^&#123;ij&#125;&#41; \cdot &#40;I - \lambda \delta^&#123;ij&#125;&#41; &#61;
I^2 &#43; \lambda \delta^&#123;ij&#125;&#41;- \lambda \delta^&#123;ij&#125; - \lambda^2 \delta^&#123;ij&#125; &#61; I.
~&#36;

<p>&#40;The &#36;lk&#36; term of &#36;\delta^&#123;ij&#125; \cdot \delta^&#123;ij&#125;&#36; is &#36;\sum_m \delta^&#123;ij&#125;&#40;lm&#41; \cdot \delta^&#123;ij&#125;&#40;mn&#41;&#36;. The first means &#36;m&#61;j&#36;, the second means &#36;m&#61;i&#36;, so this is &#36;0&#36; unless &#36;i&#61;j&#36;, but &#36;i &lt; j&#36;.&#41;</p>

<p>So the inverses are lower triangular.</p>

<p>Finally, &#36;R_p R_m &#61; &#40;I &#43; \lambda \delta^&#123;ij&#125;&#41;&#40;I &#43; \gamma \delta^&#123;kl&#125;&#41; &#61; I &#43;  \lambda \delta^&#123;ij&#125; &#43; \gamma \delta^&#123;kl&#125;&#36; which is lower triangular, so the product of the inverses will be.</p>

<p>Thus &#36;L&#36; is unit lower triangular, as the diagonal is all 1.</p>

<h2>Pivoting</h2>

<p>The book points out that these two problems are similar mathematically, but not computationally:</p>

&#36;~
A &#61; \left&#91;\begin&#123;array&#125;&#123;cc&#125;
\epsilon &amp; 1\\
1 &amp; 1
\end&#123;array&#125;\right&#93;
~&#36;

<p>and</p>

&#36;~
B &#61; \left&#91;\begin&#123;array&#125;&#123;cc&#125;
1 &amp; 1\\
\epsilon &amp; 1
\end&#123;array&#125;\right&#93;
~&#36;

<p>Solving &#36;A&#61;&#91;1, 2&#93;&#36; we get</p>

&#36;~
\begin&#123;align&#125;
\epsilon\cdot x_1 &#43; 1\cdot x_2 &amp;&#61; 1\\
0 \cdot x_1 &#43; &#40;1 - \epsilon^&#123;-1&#125;&#41; x_2 &amp;&#61; 2 - \epsilon^&#123;-1&#125;
\end&#123;align&#125;
~&#36;

<p>So,</p>

&#36;~
\begin&#123;align&#125;
x_2 &amp;&#61; \frac&#123;2 - \epsilon^&#123;-1&#125;&#125;&#123;1 - \epsilon^&#123;-1&#125;&#125; \approx 1\\
x_1 &amp;&#61; \epsilon^&#123;-1&#125; &#91; 1 - \frac&#123;2-\epsilon^&#123;-1&#125;&#125;&#123;1-\epsilon^&#123;-1&#125;&#125;&#93;
\end&#123;align&#125;
~&#36;

<p>If &#36;\epsilon&#36; is small, then &#36;1/\epsilon&#36; is big and subtracting from 1 or 2 can be an issue. &#40;e.g., we saw <code>10^30 -1 &#61; 10^30</code> on the computer. So, in the computation of &#36;x_1&#36;, we might get &#36;1/\epsilon &#91;1 - 1&#93; &#61; 0&#36;. But, in fact, this is &#36; -\epsilon^&#123;-1&#125;/&#40;1-\epsilon^&#123;-1&#125;&#41; \approx 1&#36;.</p>

<p>Whereas, with &#36;Bx &#61; &#91;2; 1&#93;&#36;, we get </p>

&#36;~
\begin&#123;align&#125;
1\cdot x_1 &#43; 1\cdot x_2 &amp;&#61; 2\\
0 \cdot x_1 &#43; &#40;1 - \epsilon&#41; x_2 &amp;&#61; 2 - \epsilon
\end&#123;align&#125;
~&#36;

<p>So, &#36;x_2 &#61; &#40;1-2\epsilon&#41;/&#40;1 - \epsilon&#41; \approx 1&#36; and &#36;x_1 &#61; 2 - x_2 \approx 2 - 1 &#61; 1&#36;, as desired.</p>

<p>So, switching rows can make a <em>big</em> difference in the output when dealing with floating point.</p>

<p>&#40;We don&#39;t really switch, but rather keep track of which row is used to pivot using a permutation. Moving data in a matrix is costly, much more so than keeping track of indices.&#41;</p>

<h3>Pivoting strategies</h3>

<p>Pivoting results in  permutation of the rows of &#36;A&#36;. This can be written different ways. If &#36;P&#36; is a permutation matrix, then we can write &#36;LU &#61; PA&#36; as a &#36;LU&#36; factorization. Whereas, if <code>p</code> is a permutation vector, we can use the matrix notation of <code>julia</code> to write <code>L*U &#61; A&#91;p,:&#93;</code>.</p>

<p>There are different ways that pivot rows can be selected:</p>

<ul>
<li><strong>partial pivoting</strong>: at step &#36;i&#36; choose the row with the largest value of &#36;|a_&#123;li|&#125;&#36;, where &#36;l&#36; runs over the rows that have <em>not</em> been used to pivot. </li>
</ul>

<ul>
<li><strong>complete pivoting</strong> consider the largest of all value values in rows not used for pivoting</li>
</ul>

<ul>
<li><strong>scaled row pivoting</strong>. Defing the scale of a row by:</li>
</ul>

&#36;~
s_i &#61; max_&#123;i \leq j \leq n&#125; |a_&#123;ij&#125;| &#61; \text&#123;largest of &#125; |a_&#123;i1&#125;|, |a_&#123;i2&#125;|, \dots, |a_&#123;in&#125;|.
~&#36;

<p>Then at step &#36;i&#36;, form the ratios: &#36;|a_&#123;li&#125;/s_l|&#36;, for &#36;l&#36; running over the rows not already used for pivoting. Choose the &#36;l&#36; with the largest ratio to be the new pivot row.</p>

<p>&#40;At step &#36;i&#36;, an easier strategy could be to just pick the row which has the largest value of &#36;|a_&#123;li&#125;|&#36; where &#36;l&#36; runs over the rows which have not been used to pivot. This has the effect that the matrix &#36;L&#36; will have entries no greater than &#36;1&#36; in absolute value.&#41;</p>

<h3>Example</h3>

<p>Consider the matrix</p>

In [ ]:
Aorig = [-1.0 1 -4; 2 3 1; 3 3 2]
A = copy(Aorig)

3x3 Array{Float64,2}:
 -1.0  1.0  -4.0
  2.0  3.0   1.0
  3.0  3.0   2.0

<p>We first do partial pivoting:</p>

<p>As &#36;3&#36; is the largest valuein the first column, we choose row 3 to pivot on, and eliminate the first entry in rows 1 and 2 by the appropriate multiplication. We store those values in a matrix &#36;L&#36;:</p>

In [ ]:
L = zeros(A) # copies size of A, fills with 0
i = 1
p = 3
ps = [p]
L[ps[end], i] = 1
for l in setdiff(1:3, ps)
   L[l, i] = A[l, i] / A[p, i]
   A[l, :] = A[l,:] - L[l,i]*A[p,:]
 end
A   

3x3 Array{Float64,2}:
 0.0  2.0  -3.33333 
 0.0  1.0  -0.333333
 3.0  3.0   2.0     

<p>We now have to consider the matrix without the third row:</p>

In [ ]:
A[setdiff(1:3, ps), :]

2x3 Array{Float64,2}:
 0.0  2.0  -3.33333 
 0.0  1.0  -0.333333

<p>We see that row 1 is the next pivot row, as &#36;2&#36; is bigger that &#36;1&#36; in absolute value</p>

In [ ]:
i = 2
p = 1
ps = push!(ps, p)
L[ps[end], i] = 1
for l in setdiff(1:3, ps)
   L[l, i] = A[l, i] / A[p, i]
   A[l, :] = A[l,:] - L[l,i]*A[p,:]
 end
A   

3x3 Array{Float64,2}:
 0.0  2.0  -3.33333
 0.0  0.0   1.33333
 3.0  3.0   2.0    

<p>We finish by pushing the last row, &#36;2&#36; onto our permutation vector and making sure to set the diagonal entry to &#36;1&#36;:</p>

In [ ]:
i, p = 3, 2
push!(ps, p)
L[p,i] = 1

1

<p>Are we done? Our bookkeeping, <code>ps</code>, points to the pivot rows, to see &#36;L&#36; and &#36;U&#36; we need to reverse that:</p>

In [ ]:
L = L[ps, :]
U = A[ps, :]
L, U

(
3x3 Array{Float64,2}:
  1.0       0.0  0.0
 -0.333333  1.0  0.0
  0.666667  0.5  1.0,

3x3 Array{Float64,2}:
 3.0  3.0   2.0    
 0.0  2.0  -3.33333
 0.0  0.0   1.33333)

<p>And we verify</p>

In [ ]:
L*U - Aorig[ps, :]

3x3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

<p>To compare below, we record the 3 values computed:</p>

In [ ]:
L, U, ps

(
3x3 Array{Float64,2}:
  1.0       0.0  0.0
 -0.333333  1.0  0.0
  0.666667  0.5  1.0,

3x3 Array{Float64,2}:
 3.0  3.0   2.0    
 0.0  2.0  -3.33333
 0.0  0.0   1.33333,

[3,1,2])

<h3>Scaled row pivoting</h3>

<p>Now we repeat with scaled row pivoting:</p>

In [ ]:
A = copy(Aorig)

3x3 Array{Float64,2}:
 -1.0  1.0  -4.0
  2.0  3.0   1.0
  3.0  3.0   2.0

<p>We begin by finding the scale for each row:</p>

In [ ]:
s = mapslices(x->maximum(abs(x)), A, 2) # mapslices does all rows at once

3x1 Array{Float64,2}:
 4.0
 3.0
 3.0

<p>Then we divide column 1 to find the largest:</p>

In [ ]:
i = 1
A[:,i] ./ s

3x1 Array{Float64,2}:
 -0.25    
  0.666667
  1.0     

<p>We see &#36;p&#61;3&#36; will work.</p>

In [ ]:
L = zeros(A)  
p=3
ps = [p]
L[p,i] = 1
for l in setdiff(1:3, ps)
  L[l,i]=  (A[l,i]/A[p,i])
  A[l,:] = A[l,:] - L[l,i]*A[p,:]
end
A

3x3 Array{Float64,2}:
 0.0  2.0  -3.33333 
 0.0  1.0  -0.333333
 3.0  3.0   2.0     

<p>Now we repeat, only we don&#39;t want to consider row 3, which we keep track of mentally and don&#39;t try to code.</p>

In [ ]:
s = mapslices(x->maximum(abs(x)), A, 2) 

3x1 Array{Float64,2}:
 3.33333
 1.0    
 3.0    

In [ ]:
i = 2
A[:,i]./ s

3x1 Array{Float64,2}:
 0.6
 1.0
 1.0

<p>The answer is &#36;p&#61;2&#36; now. We  continue by subtracting a multiple of row 2 from row 1 </p>

In [ ]:
p = 2
push!(ps, p)
L[p, i] = 1
L[1,i] = A[1,i] /A[p,i]
A[1,:] = A[1,:] - L[1,i] * A[p,:]
A

3x3 Array{Float64,2}:
 0.0  0.0  -2.66667 
 0.0  1.0  -0.333333
 3.0  3.0   2.0     

<p>So we have our permutation vector: &#36;p &#61; &#91;3, 2, 1&#93;&#36;. Our matrix &#36;U&#36; is then seen to be</p>

In [ ]:
push!(ps, 1)
L[1,3] = 1
U = A[ps,  :]

3x3 Array{Float64,2}:
 3.0  3.0   2.0     
 0.0  1.0  -0.333333
 0.0  0.0  -2.66667 

<p>Our matrix &#36;L&#36; comes from using the  factors above &#40;where we used minus signs&#41;:</p>

In [ ]:
L = L[ps, :]

3x3 Array{Float64,2}:
  1.0       0.0  0.0
  0.666667  1.0  0.0
 -0.333333  2.0  1.0

<p>Finally, we check that we got what we wanted: &#36;LU&#36; is the permuted &#36;A&#36;:</p>

In [ ]:
L*U - Aorig[ps,:]

3x3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

<p>For scaled pivoting we found:</p>

In [ ]:
L, U, ps

(
3x3 Array{Float64,2}:
  1.0       0.0  0.0
  0.666667  1.0  0.0
 -0.333333  2.0  1.0,

3x3 Array{Float64,2}:
 3.0  3.0   2.0     
 0.0  1.0  -0.333333
 0.0  0.0  -2.66667 ,

[3,2,1])

<p>This is different than partial pivoting, as might be expected in some cases. The &#36;LU&#36; factorization is not unique. What can be said is:</p>

<blockquote>
<p>Theorem 2, p173 With pivoting, there is a factorization &#36;LU &#61; PA&#36;.</p>
</blockquote>

<h3>Compare to lu function</h3>

<p>Julia has a built in <code>lu</code> function to return the triple:</p>

In [ ]:
A = copy(Aorig)
lu(A)

(
3x3 Array{Float64,2}:
  1.0       0.0  0.0
 -0.333333  1.0  0.0
  0.666667  0.5  1.0,

3x3 Array{Float64,2}:
 3.0  3.0   2.0    
 0.0  2.0  -3.33333
 0.0  0.0   1.33333,

[3,1,2])

<p>This agrees with the factorization we found with partial pivoting. The empirical claim is that partial pivoting is nearly always better compared to complete pivoting due to the extra overhead required for complete pivoting.</p>

<h2>Counting steps</h2>

<p>Suppose &#36;A&#36; is &#36;n \times n&#36;. If applicable, mathematicallly solving &#36;Ax &#61; b&#36; is as easy as writing &#36;x &#61; A^&#123;-1&#125; b&#36;. Why bother with &#36;LUx &#61; b&#36; and then solving both &#36;Ly&#61;b&#36; and &#36;Ux&#61;y&#36;? &#40;Actually we take  &#36;&#91;A ~b&#93;&#36; and find &#36;&#91;U ~ b&#39;&#93;&#36; then solve this backsubstitution problem.&#41;</p>

<p>One reason is it takes fewer steps.</p>

<p>The book argues that addition and subtraction are much cheaper to do than multiplication and division, so when counting operations, we should just count the long operations, <code>ops</code> or short.</p>

<p>Let&#39;s look at how many such operations are done for our pivoted Gaussian elimination. First the &#36;U&#36; term.</p>

<p>For &#36;i&#61;1&#36;, the first row is multiplied by some factor and &#36;n-1&#36; times and subtracted from other rows. This is &#36;n\cdot&#40;n-1&#41;&#36; steps, not counting the division to find the factor. For that, we have &#36;n&#36; operations to find the pivot row &#40;not counting the time to find a maximum, we did divide the &#36;i&#36;th column by the scale, hence &#36;n&#36;&#41;. So all told, we have basically &#36;n&#40;n-1&#41; &#43; n&#36; <code>ops</code>.</p>

<p>For &#36;i&#61;2&#36;, we could repeat, but note that we are basically working with matrix without the &#36;p&#36;th row and the first column, so we have the same computation with an &#36;&#40;n-1&#41; \times &#40;n-1&#41;&#36; size matrix. So the answer will be &#36;n-1&#36; <code>ops</code>.</p>

<p>And so on, to where we get this elimination costs us this many <code>ops</code>:</p>

&#36;~
n^2 &#43; &#40;n-1&#41;^2 &#43; \cdots &#43; 2^2 &#61; \sum_&#123;i&#61;1&#125;^n i^2 -1 &#61; \frac&#123;n&#40;n-1&#41;&#40;2n-1&#41;&#125;&#123;6&#125; -1 \approx \frac&#123;n^3&#125;&#123;3&#125; &#43; \frac&#123;n^2&#125;&#123;2&#125;.
~&#36;

<p>Now to count the steps to modify the &#36;b&#36; to get &#36;b&#39;&#36;. The first row introduces &#36;n-1&#36;, the second &#36;n-2&#36;, the third &#36;n-3&#36; and so on, so this involves &#36;\sum_&#123;i&#61;1&#125;^&#123;n-1&#125; u&#36;  or &#36;n&#40;n-1&#41;/2&#36;</p>

<p>To do the back substitution, we have &#36;1&#36; division for the first row, &#36;2&#36; <code>ops</code> for the second &#40;we have &#36;ax_j &#43; bx_&#123;j-1&#125;&#61;c&#36; and we solve for &#36;x_j&#36; through &#36;&#40;1/a&#41; \cdot &#40;c - bx_&#123;j-1&#125;&#41;&#36; which is &#36;2&#36; <code>ops</code>.&#41; And so on to give: &#36; 1 &#43;2 &#43; \cdots &#43;n&#36; or &#36;&#40;n&#43;1&#41;n/2&#36; counts.</p>

<p>All told we can put together to get a count. The book presents this a bit differently, for the case where we are solving &#36;&#91;A; b_1;b_2; \dots;b_m&#93;&#36; simulataneously, and then we have:</p>

<blockquote>
<p>Theorem 4 &#40;p176&#41; on Long Operations: To solve &#36;Ax&#61;b&#36; for &#36;m&#36; vectors &#36;b&#36; where &#36;A&#36; is &#36;n\times n&#36; involves approximately this many <code>ops</code>:</p>
</blockquote>

&#36;~
\frac&#123;n^3&#125;&#123;3&#125; &#43; &#40;\frac&#123;1&#125;&#123;2&#125; &#43; m&#41; n^2.
~&#36;

<p>So, for a single &#36;b&#36;, to solve &#36;Ax&#61;b&#36; by this method is basically &#36;n^3/3&#36; steps. Whereas to find the inverse itself &#40;and not counting the steps in &#36;A^&#123;-1&#125;b&#36;, can be found by taking &#36;m&#61;n&#36; in the formula &#40;reducing &#36;&#91;A;I&#93;&#36;&#41;. To give:</p>

&#36;~
\text&#123;steps to find &#125; A^&#123;-1&#125; &#61; \frac&#123;n^3&#125;&#123;3&#125; &#43; &#40;\frac&#123;1&#125;&#123;2&#125; &#43; n&#41; n^2 \approx \frac&#123;n^3&#125;&#123;3&#125; &#43; n^3 &#61; \frac&#123;4&#125;&#123;3&#125; n^3.
~&#36;

<h2>Special case – Diagonally dominant matrices</h2>

<p>Call the matrix &#36;A&#36; *diagonally dominant&#36; if the diagonal terms are the largest in each row and column. &#40;That is &#36;|a_&#123;ii&#125;| \geq |a_&#123;il&#125;|, |a_&#123;ki&#125;|&#36;&#41;.</p>

<blockquote>
<p>Thm: A diagonal dominant matrix is non-singular and has an LU factorization where no pivoting is necessary.</p>
</blockquote>

<p>This is a result of the fact that Gaussian elimination preserves the diagonal dominance, so picking a different partial pivot row or scaled pivot row is unecessary.</p>

<h2>tridiagonal system</h2>

<p>Consider the approximation formula for the second derivative:</p>

&#36;~
f&#39;&#39;&#40;x&#41; \approx \frac&#123;f&#40;x &#43; h&#41; - 2 f&#40;x&#41; &#43; f&#40;x-h&#41;&#125;&#123;h^2&#125;
~&#36;

<p>If we took an interval, say &#36;&#91;0,1&#93;&#36; and split into a grid of &#36;n&#43;1&#36; points, then we could discretize &#36;f&#36; on these points. Using just these point, then we would approximate the second derivative at each point as above. Suppose we had values &#36;f_i &#61; f&#40;&#40;i-1_/n&#41;&#36; for &#36;i &#61; 1, 1, \dots n&#43;1&#36;. Then we could do many of these derivatives at once using matrix notation. To be speicific, we take &#36;n&#61;4&#36;. Then we get</p>

In [ ]:
n=4
xs = linspace(0, 1, n)

linspace(0.0,1.0,4)

In [ ]:
f(x) = sin(x)
fs = map(f, xs)

4-element Array{Float64,1}:
 0.0     
 0.327195
 0.61837 
 0.841471

In [ ]:
A =  [-2 1 0 0; 1 -2 1 0; 0 1 -2 1; 0 0 1 -2]

4x4 Array{Int64,2}:
 -2   1   0   0
  1  -2   1   0
  0   1  -2   1
  0   0   1  -2

<p>And then, we would have</p>

In [ ]:
h = (1-0)/n
(A * fs) / h^2 + fs

4-element Array{Float64,1}:
   5.23512 
  -0.249119
  -0.470813
 -16.1917  

<p>We see we don&#39;t have very good approximations to the second derivative. Were we to increase &#36;n&#36; this could be the case, except on the edge cases:</p>

In [ ]:
n = 1000
xs = linspace(0, 1, n)
fs = map(f, xs)

A = zeros(n,n)
for i in 1:n A[i,i] = -2 end
for i in 2:n
  A[i,i-1] = 1
  A[i-1, i] = 1
end
h = (1-0)/n
(A * fs) / h^2 + fs

1000-element Array{Float64,1}:
 1001.0       
   -2.00492e-6
   -4.00985e-6
   -6.01477e-6
   -8.01968e-6
   -1.00246e-5
   -1.20295e-5
   -1.40344e-5
   -1.60392e-5
   -1.80441e-5
   -2.00489e-5
   -2.20537e-5
   -2.40585e-5
   -2.60633e-5
    ⋮         
   -0.00167228
   -0.00167338
   -0.00167448
   -0.00167558
   -0.00167668
   -0.00167778
   -0.00167887
   -0.00167996
   -0.00168105
   -0.00168214
   -0.00168323
   -0.00168432
   -8.42011e5 

<p>But our focus here is on the special shape of &#36;A&#36; – it is tridiagonal. that is &#36;a_&#123;ij&#125; &#61; 0&#36; if &#36;|i-j| &gt; 1&#36;. For this system – which does not need pivoting – then number of <code>ops</code> is dramatically reduced, as each row is done with &#36;2&#36; <code>ops</code>, so to get the &#36;LU&#36; factorization can be done quite quickly.</p>

<p>This is a good thing, as we see that large &#36;n&#36;s are needed to get accuracy, an if the algorithm scaled like &#36;&#40;1/3&#41; \cdot n^3&#36; that would be a big problem.</p>